# Informations sur la base de données.

#### Buts de l'analyse

#### Étapes de traitement des données

#### Importation des librairies utilisées.

In [ ]:
import os

import sklearn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import scipy.stats as stats

%matplotlib inline
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# Pour la reproductibilité des résultats
np.random.seed(43)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('Do not show this message')

# some nice colors from http://colorbrewer2.org/
COLOR1 = '#7fc97f'
COLOR2 = '#beaed4'
COLOR3 = '#fdc086'
COLOR4 = '#ffff99'
COLOR5 = '#386cb0'

sns.set(rc={'figure.figsize':(9, 4)})
sns.set_theme(context="notebook", style="white", palette="Set2")

## Recommandations 

### Analyse des forme


* **Ligne et colonne: (6498063, 7)**

niveau de detail, une ligne par :
                  Géographie, 
                  Classification nationale des professions(CNP4),
                  Sexe,
                  Durée de l'emploi,	
                  Période de référence, mensuelle

* **Type de variables: 2 variables de type numérique et 5 variables qualitative.**

    * Géographie : Canada, Province ou territoire
    * Classification nationale des professions(CNP4)
    * Sexe
    * Mois: nombre moyenne de la durée d'emploi
    * Personnes : le nombre des personnes 
    * Période de référence : mensuelle

La repartition des individus est egale pour toutes les modalites des variables Sexe, CNP, Géographie,.Périoade de référence 

* La variable Regions contient 11 modalites
* La variable CNP contient 51 modalites
* La variable Sexe contienttient 3 modaliteses
* La variable Durée de l'emploi contient 9 modalites
* La variable Année/Mois contient 429 modalites


    
* **Valeurs extremes **

On va supposer, pour l'exemple, que la base de données contient 0,5% de valeurs aberrantes de toutes sortes. Ce n'est pas exagéré. Cela représente $0,005*168300 \approx 841$ valeurs.

On utilise la méthode de la [forêt d'isolation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html) pour détecter et éliminer un maximum de 0,5% des valeurs possiblement aberrantes.  
    
### Analyse de structure

* Conserver seulement les variables :  
    * Géographie : Canada, Province ou territoire,
    * Classification nationale des professions(CNP4)
    * Sexe
    * Personnes
    * Duree moyenne d'emploi
    * Période de référence : annuelle


### Nettoyage des donnees: Recommandations

* Filtrer la modalite "Les deux sexes" de la variable **Sexe**
* Filtrer la modalite "'Canada'" de la variable **Region**
* Agreger l'information pour avoir une fréquence annuelle 
* Plage temporelle : 1997-2021



# Partie I: Prétraitement des données.

## Lecture des bases de données en format CSV.

Les données sont écrites dans un DataFrame de la librairie Pandas.

In [ ]:
cwd=r"D:\Daiana\Documents\AEC Intelligence artificielle\Mise en place d'un écosystème d'IA\salary\data\interim\duree_emploi_CNP"

In [ ]:
def load_db():
    duree_emploi_CNP=pd.read_csv(cwd,sep=",")
    return duree_emploi_CNP#,postes_vacants_CNP,caractéristiques_population_active,heures_habituellement_travaille,heures_effectivement_travaille,personnes_avec_plus_un_emploi,raison_quitter_emploi,absence_travail_temps_plein

In [ ]:
duree_emploi_CNP=load_db()

## Explorer les données

In [ ]:
duree_emploi_CNP.head()

In [ ]:
duree_emploi_CNP.tail()

### Dimensions du dataset

In [ ]:
print("Les dimensions du jeux des donnees: ", duree_emploi_CNP.shape)
print() 

In [ ]:
# Size of our data set
## le nombre de lignes multiplié par le nombre de colonnes si DataFrame.
print("Nombre d’éléments:", duree_emploi_CNP.size)
print()

Le nombre d'individus est égale à 6498063 et le nombre de descripteurs est égale à 7

### Type des donnees

In [ ]:
duree_emploi_CNP.dtypes

In [ ]:
duree_emploi_CNP.dtypes.value_counts()

Le jeu des données a 2 variables de type numérique et 5 variables qualitative.

In [ ]:
duree_emploi_CNP.describe()

In [ ]:
duree_emploi_CNP.info()

### Affichage du nombre de valeurs manquantes Nans dans le DataFrame.

In [ ]:
duree_emploi_CNP.isna().sum()

In [ ]:
duree_emploi_CNP[duree_emploi_CNP.Mois.notnull()]["Durée de l'emploi"].value_counts()

In [ ]:
duree_emploi_CNP[duree_emploi_CNP.Mois.notnull()]

### Pivoter la Duree de l emploi

In [ ]:

df1_=duree_emploi_CNP[duree_emploi_CNP.Mois.notnull()][["Annee/Mois","Regions","CNP","Sexe","Durée de l'emploi","Mois"]].set_index(["Annee/Mois","Regions","CNP","Sexe"])

In [ ]:
df1_.columns

In [ ]:
duree_emploi_CNP[duree_emploi_CNP.Mois.isnull()]["Durée de l'emploi"].value_counts()

In [ ]:
df2_=duree_emploi_CNP[duree_emploi_CNP.Mois.isnull()][["Annee/Mois","Regions","CNP","Sexe","Durée de l'emploi","Personnes"]].set_index(["Annee/Mois","Regions","CNP","Sexe"])

In [ ]:
df2_

In [ ]:
duree_emploi_CNP_=df1_.merge(df2_,how="inner",left_index=True, right_index=True).reset_index()

In [ ]:
duree_emploi_CNP_

In [ ]:
duree_emploi_CNP_.rename(columns={"Mois":"Duree moyenne emploi","Durée de l'emploi_y":"Durée de l'emploi"},inplace=True)

In [ ]:
duree_emploi_CNP_=duree_emploi_CNP_.drop("Durée de l'emploi_x",axis=1)

In [ ]:
duree_emploi_CNP_.shape

### Analyse caractéristiques catégorielles

#### Séparer les données en colonnes numériques et catégorielle

In [ ]:
cat_col=[]
num_col=[]

for col in duree_emploi_CNP_.columns:
    if duree_emploi_CNP_[col].dtypes=="object":
        cat_col.append(col)
    else:
        num_col.append(col)
print ("Variable catégorielle:",cat_col)
print("Variable numériques:",num_col)

#### Nombre valeurs uniques

In [ ]:
for col in cat_col:
    print(col,'({})'.format(duree_emploi_CNP_[col].nunique()),":")
    print(duree_emploi_CNP_[col].value_counts())
    print()
    print()

* La variable Regions contient 11 modalites
* La variable CNP contient 51 modalites
* La variable Sexe contienttient 3 modaliteses
* La variable Durée de l'emploi contient 9 modalites
* La variable Année/Mois contient 429 modalites


Note: La répartition des données est égale pour toutes les modalités de chaque catégories

### Analyse caractéristiques numériques

In [ ]:
duree_emploi_CNP_.describe()

La durée moyenne de travail est de 96 mois (~ 8 ans).
Au moins 75% des personnes ont travailler 119.5 mois (~ 10 ans)

In [ ]:
duree_emploi_CNP_0=duree_emploi_CNP_[duree_emploi_CNP_["Duree moyenne emploi"]==0.0]

In [ ]:
(duree_emploi_CNP_[duree_emploi_CNP_["Duree moyenne emploi"]==0.0].shape[0]/duree_emploi_CNP_.shape[0])*100

Il y a 4% (227.184) occurences dont les variables duree moyenne emploi, et personnes ont la valeur 0.0 

In [ ]:
def process_date(dg):
    return pd.DataFrame([[
                        dg["Personnes"].sum(),
                        dg["Duree moyenne emploi"].mean(),
                        ]], columns=["Personnes", "Duree moyenne emploi"])

duree_emploi_CNP_0.groupby(["Regions","CNP","Sexe","Durée de l'emploi","Annee/Mois"]).apply(process_date).reset_index(4, drop=True).reset_index(drop=False)

### Agregation par année

In [ ]:
duree_emploi_CNP_['Annee'] = duree_emploi_CNP_['Annee/Mois'].str[:4]

In [ ]:
duree_emploi_CNP_.head()

In [ ]:
def process_date(dg):
    return pd.DataFrame([[
                        dg["Personnes"].sum(),
                        dg["Duree moyenne emploi"].mean(),
                        ]], columns=["Personnes", "Duree moyenne emploi"])



In [ ]:
df_duree_emploi=duree_emploi_CNP_.groupby(["Regions","CNP","Sexe","Durée de l'emploi","Annee"]).apply(process_date).reset_index(5, drop=True).reset_index(drop=False)

In [ ]:
df_duree_emploi.describe()

## Visualisation des données

Avec les année la difference homme vs femme de la durée d'emploi a diminuer.  

In [ ]:
filtre_sexe=df_duree_emploi['Sexe'].isin(['Femmes','Hommes'])
sns.lineplot(data=df_duree_emploi[filtre_sexe], 
             x="Annee", 
             y='Duree moyenne emploi',
             hue='Sexe',
            markers=True,
            dashes=False)
plt.title(label="Evolution de la durée d'emploi homme vs femme,1987-2022 (Source: Donnees StatCanada)",
          fontdict={'fontsize':10}, loc='center')
plt.xticks(rotation=75)
plt.show()

In [ ]:
filtre_sexe_Ont_Qc=(df_duree_emploi['Sexe'].isin(['Femmes','Hommes'])) & (df_duree_emploi['Regions'].isin(['Ontario','Québec']))
sns.relplot(data=df_duree_emploi[filtre_sexe_Ont_Qc],
             x="Annee", 
             y='Duree moyenne emploi',
             hue='Sexe',
             col='Regions',
             kind='line',
             #markers=True, 
            dashes=True
            
            )
plt.title(label='Evolution du salaire horaire moyenne homme vs femme,1997-2021 (Source: Donnees StatCanada)',
          fontdict={'fontsize':10}, loc='left',y=1.1)
plt.xticks(rotation=75)
plt.show()

### Distribution des effectifs par region, region /sexe

In [ ]:
#sal_empl_sexe=salaire_employe_CNP['Sexe'].isin(['Femmes','Hommes'])
filtre_sexe=df_duree_emploi['Sexe'].isin(['Femmes','Hommes'])

plt.figure(figsize=(8,12))
sns.catplot(data=df_duree_emploi[filtre_sexe], x="Regions", 
            y="Duree moyenne emploi", 
            kind="box",
            hue='Sexe',
            #palette="pastel",
           dodge=True)
plt.title(label="Distribution de la durée moyenne d'emploi selon regions et sexe,donnees 1987-2022(source Statistique Canada)",
          fontdict={'fontsize':10}, loc='center')
plt.xticks(rotation=75)
plt.show()

In [ ]:
df_duree_emploi.columns

In [ ]:
df_duree_emploi["Durée de l'emploi"].unique()

## Filtres

* Filtrer la modalite "Les deux sexes" de la variable **Sexe**
* Filtrer la modalite "'Canada'" de la variable **Region**

In [ ]:
def filtre_df():
    filtre_sexe=df_duree_emploi.Sexe.isin(["Femmes","Hommes"])
    filtre_region=df_duree_emploi["Regions"]!="Canada"
    filtre_annee=(pd.Series(df_duree_emploi["Annee"]).between("1997","2021"))
    
    df1=df_duree_emploi[filtre_sexe]
    df2=df1[filtre_region]
    df=df2[filtre_annee]
    
    return df

In [ ]:
df_duree_emploi=filtre_df()

In [ ]:
df_duree_emploi.shape

## Exporter to csv

In [ ]:
df_duree_emploi.to_csv(r"D:\Daiana\Documents\AEC Intelligence artificielle\Mise en place d'un écosystème d'IA\salary\data\raw\df_duree_emploi", index = False)